In [12]:
from scipy.io import wavfile
import scipy.signal as sps
import os
import librosa
import pandas as pd
import numpy as np
import json

import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [13]:
def resample(path,new_rate = 16000):
    sampling_rate, data = wavfile.read(path)
    samples = round(len(data) * float(new_rate) / sampling_rate)
    new_data = sps.resample(data, samples)
    return new_data

In [14]:
def js_to_np(path,max_len=50):    
    f = open(path)
    data = json.load(f)
    t=[[j['e'] for j in i['l']] for i in data]
    t=[j for i in t for j in i]
    if (len(t)>50): t=t[:50]
    return np.pad(t,(0,50-len(t)))

In [15]:
def inp():
    ds=["data/train/songs/"+f for f in os.listdir("data/train/songs/") if os.path.isfile(os.path.join("data/train/songs/", f))]
    for i in ds:
        label=i.replace("songs","labels").replace("wav","json")
        t=np.mean(resample(i),axis=1)
        yield (librosa.feature.melspectrogram(y=np.pad(t,(0,480000-len(t))),sr=16000),js_to_np(label))
x=[]
y=[]
for i in inp():
    x.append(i[0])
    y.append(i[1])
x=np.array(x)
y=np.array(y)
with open('test.npy', 'wb') as f:
    np.save(f, x)
    np.save(f, y)

KeyboardInterrupt: 

In [16]:
with open('test.npy', 'rb') as f:
    x = np.load(f)
    y = np.load(f)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1028)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1028,activation = 'relu'),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(50,activation = 'relu')
])
model.compile(loss="mse",optimizer='adam',metrics=["mae"])
model.fit(x=x,y=y, batch_size=10, epochs=50)

Epoch 1/50
106/106 [==============================] - 13s 89ms/step - loss: 49712384.0000 - mae: 5108.3623
Epoch 2/50
106/106 [==============================] - 9s 88ms/step - loss: 36230064.0000 - mae: 4445.8125
Epoch 3/50
106/106 [==============================] - 9s 89ms/step - loss: 36047228.0000 - mae: 4450.9966
Epoch 4/50
106/106 [==============================] - 9s 88ms/step - loss: 34306044.0000 - mae: 4324.4595
Epoch 5/50
106/106 [==============================] - 9s 89ms/step - loss: 32831882.0000 - mae: 4157.0254
Epoch 6/50
106/106 [==============================] - 10s 90ms/step - loss: 31461526.0000 - mae: 4048.9214
Epoch 7/50
106/106 [==============================] - 9s 88ms/step - loss: 29311380.0000 - mae: 3824.1697
Epoch 8/50
106/106 [==============================] - 9s 89ms/step - loss: 28682208.0000 - mae: 3730.7014
Epoch 9/50
106/106 [==============================] - 9s 89ms/step - loss: 28170826.0000 - mae: 3621.8887
Epoch 10/50
106/106 [=======================